In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from operator import add
import sys
import operator
from pyspark.sql.functions import broadcast
from pyspark.sql.types import StringType
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1590568945548_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
year=1994
flights_file_path="s3://air-traffic-dataset/data-partitioned/ontimeperformance_flights_large_T2/year={}".format(year)

ftd = (
    spark.read.format("csv")
    .options(header="true")
    .load(flights_file_path)
)

In [ ]:
ftd.printSchema()

In [ ]:
ftd.explain()

In [2]:
def parseAvgFlights(avg_flights_df):
    results = avg_flights_df.collect()
    results.sort(key=operator.itemgetter(0))
    for i in range(len(results)):
        print(
            "{} \t {} \t {} \t {} \t {}".format(
                results[i][0],
                results[i][1],
                results[i][2],
                results[i][3],
                results[i][4],
            ),
            file=sys.stdout,
        )

def delayed_flights(spark, flights_file_path, other_files_path, year):
    """

    PARAMETERS
    ----------

    spark: SparkSession
    file:  The data file e.g."s3://air-traffic-dataset/ontimeperformance_flights_test.csv".
    year: 1994-2008 inclusive for the tiny dataset.

    """

    flights_tiny_df = (
        spark.read.format("csv")
        .options(header="true")
        .load("{}/year={}".format(flights_file_path, year))
    )

    airlines_df = (
        spark.read.format("csv")
        .options(header="true")
        .load("{}/ontimeperformance_airlines.csv".format(other_files_path))
    )

    # flight_cols = ["carrier_code", "tail_number", "delayed_time"]

    flights_tiny_df = flights_tiny_df \
        .withColumn(
            "scheduled_departure_timestamp",
            F.to_timestamp(
                F.when(
                    F.col("scheduled_depature_time") == "24:00:00", "00:00:00"
                ).otherwise(F.col("scheduled_depature_time")),
                "HH:mm:ss",
            ),
        ) \
        .withColumn(
            "actual_departure_timestamp",
            F.to_timestamp(
                F.when(F.col("actual_departure_time") == "24:00:00", "00:00:00").otherwise(
                    F.col("actual_departure_time")
                ),
                "HH:mm:ss",
            ),
        ) \
        .withColumn(
            "delayed_time",
            F.when(
                F.col("actual_departure_timestamp").cast("long")
                - F.col("scheduled_departure_timestamp").cast("long")
                > (60 * 60 * 12),
                (
                    F.col("scheduled_departure_timestamp").cast("long")
                    + (60 * 60 * 24)
                    - F.col("actual_departure_timestamp").cast("long")
                )
                / 60,
            )
            .when(
                F.col("scheduled_departure_timestamp").cast("long")
                - F.col("actual_departure_timestamp").cast("long")
                > (60 * 60 * 12),
                (
                    F.col("actual_departure_timestamp").cast("long")
                    + (60 * 60 * 24)
                    - F.col("scheduled_departure_timestamp").cast("long")
                )
                / 60,
            )
            .otherwise(
                (
                    F.col("actual_departure_timestamp").cast("long")
                    - F.col("scheduled_departure_timestamp").cast("long")
                )
                / 60
            ),
        ) \
        .filter(F.col("actual_departure_timestamp").isNotNull()) \
        .filter(F.col("delayed_time") > 0) \
        .groupBy("carrier_code") \
        .agg(
            F.count("delayed_time").alias("numOfDelays"),
            F.mean("delayed_time").alias("avgDelays"),
            F.min("delayed_time").alias("minDelay"),
            F.max("delayed_time").alias("maxDelay"),
        ) \
        .withColumn("avgDelays", F.round(F.col("avgDelays"), 2)) \
        .select(
            "carrier_code",
            "numOfDelays",
            "avgDelays",
            "minDelay",
            "maxDelay"
        ) \

    avg_flights = F.broadcast(airlines_df) \
        .join(
            flights_tiny_df,
            flights_tiny_df.carrier_code == airlines_df.carrier_code
        ) \
        .select(
            "name",
            "numOfDelays",
            "avgDelays",
            "minDelay",
            "maxDelay"
        )

    parseAvgFlights(avg_flights)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
year=1996
filename="ontimeperformance_flights_massive_T2"
flights_file_path="s3://air-traffic-dataset/data-partitioned/{}".format(filename)
other_files_path="s3://air-traffic-dataset"

delayed_flights(spark, flights_file_path, other_files_path, year)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Alaska Airlines Inc. 	 3269 	 29.76 	 1.0 	 535.0
American Airlines Inc. 	 108655 	 119.34 	 1.0 	 720.0
Continental Air Lines Inc. 	 52321 	 21.51 	 1.0 	 560.0
Delta Air Lines Inc. 	 101799 	 19.92 	 1.0 	 536.0
Northwest Airlines Inc. 	 35534 	 27.44 	 1.0 	 667.0
Southwest Airlines Co. 	 24775 	 24.05 	 1.0 	 375.0
Sparrow Aviation 	 18419 	 27.89 	 1.0 	 595.0
Tway Air 	 10279 	 33.57 	 1.0 	 471.0
US Airways 	 13413 	 17.82 	 1.0 	 455.0
United Airlines 	 145323 	 21.28 	 1.0 	 719.0

In [7]:
year="1994"
flights_file_path="s3://air-traffic-dataset/data-partitioned/ontimeperformance_flights_large_T2"
other_files_path="s3://air-traffic-dataset"

delayed_flights(spark, flights_file_path, other_files_path, year)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Alaska Airlines Inc. 	 1787 	 18.34 	 1.0 	 213.0
American Airlines Inc. 	 81351 	 15.93 	 1.0 	 660.0
Continental Air Lines Inc. 	 49785 	 20.42 	 1.0 	 534.0
Delta Air Lines Inc. 	 113621 	 11.18 	 1.0 	 705.0
Northwest Airlines Inc. 	 25480 	 19.21 	 1.0 	 718.0
Southwest Airlines Co. 	 19731 	 18.55 	 1.0 	 299.0
Sparrow Aviation 	 16055 	 19.53 	 1.0 	 568.0
Tway Air 	 9874 	 25.45 	 1.0 	 441.0
US Airways 	 21393 	 15.36 	 1.0 	 478.0
United Airlines 	 102607 	 13.9 	 1.0 	 487.0

In [ ]:
flights_file_path="s3://air-traffic-dataset/data-partitioned/ontimeperformance_flights_large_T2"

ftd_test = (
    spark.read.format("csv")
    .options(header="false")
    .load(flights_file_path)
)

ftd_test.printSchema()

In [ ]:
ftd_test = ftd_test.withColumnRenamed(
    "_c0", "flight_id",
).withColumnRenamed(
    "_c1", "carrier_code",
).withColumnRenamed(
    "_c2", "flight_number",
).withColumnRenamed(
    "_c3", "flight_date",
).withColumnRenamed(
    "_c4", "origin",
).withColumnRenamed(
    "_c5", "destination",
).withColumnRenamed(
    "_c6", "tail_number",
).withColumnRenamed(
    "_c7", "scheduled_depature_time",
).withColumnRenamed(
    "_c8", "scheduled_arrival_time",
).withColumnRenamed(
    "_c9", "actual_departure_time",
).withColumnRenamed(
    "_c10", "actual_arrival_time",
).withColumnRenamed(
    "_c11", "distance",
).withColumnRenamed(
    "_c12", "year",
)

ftd_test.printSchema()

In [ ]:
ftd_test.filter(F.col("year") == "1996")

In [ ]:
ftd_test.filter(F.col("year") == "1996").count()

In [ ]:
ftd_test.printSchema()

In [ ]:
ftd_test.columns

In [ ]:
ftd_test.filter(F.col("_c12") == 1996).show()

# TASK 3!!

In [ ]:
other_files_path="s3://air-traffic-dataset"

airlines_df = (
    spark.read.format("csv")
    .options(header="true")
    .load("{}/ontimeperformance_airlines.csv".format(other_files_path))
)

airlines_df.show()

In [ ]:
size="large"
flights_file_path="s3://usyddata3404/ontimeperformance_flights_{}.csv".format(size)
output_file_path="s3://air-traffic-dataset/data-partitioned/ontimeperformance_flights_{}_country_".format(size)

ftd = (
    spark.read.format("csv")
    .options(header="true")
    .load(flights_file_path)
)

In [ ]:
airlines_df.show()

In [ ]:
ftd_country_code = F.broadcast(airlines_df) \
    .join(ftd, airlines_df.carrier_code == ftd.carrier_code) \
    .drop(ftd.carrier_code) \
    .filter(F.col("actual_departure_time").isNotNull())
    
ftd_country_code.show()

In [ ]:
maxRow = 400000
spark.conf.set("spark.sql.files.maxRecordsPerFile", maxRow)

ftd_country_code.repartition("country","tail_number") \
    .write.csv(
    output_file_path
).option("header","true")

ftd_country_code.repartition("country") \
    .write.partitionBy("country") \
    .option("header", "true") \
    .csv("s3://air-traffic-dataset/data-partitioned/ontimeperformance_flights__T2"
)